In [2]:
from bs4 import BeautifulSoup, Tag
import json


In [7]:

def element_to_json(element):
    if isinstance(element, Tag):
        return {
            "tag": element.name,
            "attributes": dict(element.attrs),
            "children": [element_to_json(child) for child in element.contents if isinstance(child, (Tag, str))],
        }
    elif isinstance(element, str):
        text = element.strip()
        return text if text else None
    else:
        return None

def html_to_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')

    # Start from <html> tag
    root = soup.find('html')
    if not root:
        raise ValueError("No <html> root element found in the HTML file.")
    
    dom_json = element_to_json(root)
    
    # Clean up None values (empty strings or whitespace)
    def clean(node):
        if isinstance(node, dict):
            node["children"] = [clean(child) for child in node.get("children", []) if child]
            return node
        return node

    return clean(dom_json)

# Usage
if __name__ == "__main__":
    file_path = "plans.html"
    dom_structure = html_to_json(file_path)
    
    # Pretty print JSON
    print(json.dumps(dom_structure, indent=2, ensure_ascii=False))


{
  "tag": "html",
  "attributes": {},
  "children": [
    {
      "tag": "div",
      "attributes": {
        "class": [
          "container-fluid",
          "pl-0",
          "pr-0",
          "product-listing-container",
          "version1",
          "bg-soft-grey"
        ]
      },
      "children": [
        {
          "tag": "div",
          "attributes": {
            "class": [
              "container",
              "pl-0",
              "pr-0",
              "compare-plp-cards",
              "plp-compare",
              "plan-cards-list"
            ]
          },
          "children": [
            {
              "tag": "span",
              "attributes": {
                "class": [
                  "sr-only"
                ],
                "id": "dropdown-alert",
                "role": "alert",
                "data-announcementmldtext": "Multiline plans are filtered by: {sortOptionHolder}",
                "data-announcementtext": "Products are sorted by: {s

In [16]:
def summarize_dom_json(node, depth=0):
    lines = []

    def walk(node, path=""):
        if isinstance(node, dict):
            tag = node.get("tag", "unknown")
            attrs = node.get("attributes", {})
            id_attr = f" with id='{attrs['id']}'" if 'id' in attrs else ""
            class_attr = f" and classes='{', '.join(attrs['class'])}'" if 'class' in attrs else ""
            other_attrs = {
                k: v for k, v in attrs.items() if k not in ['id', 'class']
            }
            # other_str = (
            #     f" and attributes {json.dumps(other_attrs)}" if other_attrs else ""
            # )

            text_content = ""
            children = node.get("children", [])

            # Check for immediate text children
            texts = [c for c in children if isinstance(c, str)]
            if texts:
                text_content = f" containing text: \"{' '.join(t.strip() for t in texts if t.strip())}\""

            summary = f"{path}<{tag}>{id_attr}{class_attr}{text_content}".strip() #{other_str}
            lines.append(summary)

            # Recurse into children that are tags
            for i, child in enumerate(children):
                if isinstance(child, dict):
                    walk(child, path + "  ")

    walk(node)
    return "\n".join(lines)

In [18]:
print(summarize_dom_json(dom_structure))

<html>
<div> and classes='container-fluid, pl-0, pr-0, product-listing-container, version1, bg-soft-grey'
<div> and classes='container, pl-0, pr-0, compare-plp-cards, plp-compare, plan-cards-list' containing text: "Refill Heading Refresh Plan PLP Sneak Peak Carousel Promo Card Promo Card Refresh Plan PLP Disclaimers Dummy Card Start of Dummy Card End of dummy card Show More/Fewer Plans Buttons <sly data-sly-test="true" data-sly-use.showMoreFewerPlansTmpl="showmorefewerplansbtn.html" data-sly-call=""> </sly>"
<span> with id='dropdown-alert' and classes='sr-only'
<div> and classes='row, ml-20, mb-20, subtitle-three-px, text-color-black, js-refill-current-plan-info, d-none'
<div> and classes='login-text' containing text: "If you want to know the details of your current Plan, text PLAN to 611611 or click here to"
<a> with id='login' and classes='login-link, text-lowercase' containing text: "Login"
<h2> and classes='row, ml-20, mb-20, subtitle-three-px, text-color-black, font-weight-bold, d

In [23]:
from bs4 import BeautifulSoup
from collections import defaultdict
import re

def parse_html_chunks(html: str):
    soup = BeautifulSoup(html, "html.parser")
    chunks = []
    current_chunk = []

    def flush_chunk():
        nonlocal current_chunk
        if current_chunk:
            chunks.append("\n".join(current_chunk))
            current_chunk = []

    # Helper to determine if this is a "heading-ish" or "start of section" element
    def is_new_section(tag):
        if tag.name in ['h1', 'h2', 'h3', 'h4']:
            return True
        if tag.name == 'div' and any(kw in " ".join(tag.get('class', [])) for kw in ['heading', 'title', 'section', 'card']):
            return True
        return False

    # Traverse top-level tags
    for tag in soup.body.find_all(recursive=False) if soup.body else soup.find_all(recursive=False):
        # Skip script/style
        if tag.name in ['script', 'style']:
            continue

        text = tag.get_text(strip=True)
        if not text:
            continue

        # Start new chunk if this is a new section
        if is_new_section(tag):
            flush_chunk()
        current_chunk.append(f"[{tag.name.upper()}] {' '.join(tag.get('class', []))} :: {text}")

    flush_chunk()
    return chunks

In [24]:


# --- Example Usage ---
with open("plans.html", "r", encoding="utf-8") as file:
    html_data = file.read()

chunks = parse_html_chunks(html_data)
for i, chunk in enumerate(chunks, 1):
    print(f"\n--- Chunk #{i} ---\n{chunk}")
    print("\n\n")


--- Chunk #1 ---
[HTML]  :: If you want to know the details of your current Plan, text PLAN to 611611 or click here toLoginKeep planPick your perfect planBuy more, save more on America's most reliable 5G network¹See one citation below for more information about planPayment Frequency1 month1 month3 month6 month12 monthHow many lines do you need?11 LINE2 lines3 lines4 lines5 linesBest Value!My current planPlatinum Unlimited$65/mo/per linePrice is 65 dollars and 0 cents per month$60/mo for the first 3 months with Auto PayPrice is #planPriceDollar dollars and #planPriceCent cents per monthprice is dollar #priceDollar and #priceCent cents /moWas priced at #listPriceDollar dollars and #listPriceCent cents now priced at #priceDollar dollars and #priceCent cents	 /mo(0)0 star reviews from 0 customer$60 per month for the first 3 months with Auto Pay/mo/mo/month5G ultra wide4Xfaster than 5G*Unlimiteddata, talk & textUnlimitedhotspot data100 GBcloud storageDevice ProtectionInternational calling 

In [4]:
chunk_memory = """[HTML]  :: If you want to know the details of your current Plan, text PLAN to 611611 or click here toLoginKeep planPick your perfect planBuy more, save more on America's most reliable 5G network¹See one citation below for more information about planPayment Frequency1 month1 month3 month6 month12 monthHow many lines do you need?11 LINE2 lines3 lines4 lines5 linesBest Value!My current planPlatinum Unlimited$65/mo/per linePrice is 65 dollars and 0 cents per month$60/mo for the first 3 months with Auto PayPrice is #planPriceDollar dollars and #planPriceCent cents per monthprice is dollar #priceDollar and #priceCent cents /moWas priced at #listPriceDollar dollars and #listPriceCent cents now priced at #priceDollar dollars and #priceCent cents	 /mo(0)0 star reviews from 0 customer$60 per month for the first 3 months with Auto Pay/mo/mo/month5G ultra wide4Xfaster than 5G*Unlimiteddata, talk & textUnlimitedhotspot data100 GBcloud storageDevice ProtectionInternational calling and textingto Canada & Mexico30 daysof serviceUnlimited Data see star below for more informationUnlimited Data*Unlimited talk, text and data20 GB hotspot data per month100 GB cloud storageInternational calling to 60+ countriesMobile ProtectWalmart+ Membership Included*Refer to the latestTerms and conditions of service.Membership Included!What's IncludedFree delivery from storeFree shipping, no order minimumParamount + video streamingEarly access to promotions and eventsSavings on fuelMobile Scan and GoReturns from homePluto ad-free video streamingWalmart+ TravelMore ways to save$100anniversary device credit$35$45Straight Talk home internet planSELECT PLANSELECT PLANGet Broadband FactsWANT TO SEE ALL OUR PLANS?Find the right phone plan for you.GO TO PLANS PAGE5G Ultra WideMy current planGold Unlimited$55/mo/per linePrice is 55 dollars and 0 cents per month$50/mo for the first 3 months with Auto PayPrice is #planPriceDollar dollars and #planPriceCent cents per monthprice is dollar #priceDollar and #priceCent cents /moWas priced at #listPriceDollar dollars and #listPriceCent cents now priced at #priceDollar dollars and #priceCent cents	 /mo(544)4.0 star reviews from 544 customer$50 per month for the first 3 months with Auto Pay/mo/mo/month5G ultra wide4Xfaster than 5G*Unlimiteddata, talk & text30 GBhotspot data100 GBCloud StorageInternational calling and textingto Canada & Mexico30 daysof serviceUnlimited Data see star below for more informationUnlimited Data*Unlimited talk, text and data30 GB hotspot data per month100 GB cloud storageInternational calls to Canada & MexicoWalmart+ Membership Included*Refer to the latestTerms and conditions of service.Membership Included!What's IncludedFree delivery from storeFree shipping, no order minimumParamount + video streamingEarly access to promotions and eventsSavings on fuelMobile Scan and GoReturns from homePluto ad-free video streamingWalmart+ TravelMore ways to save$100anniversary device credit$35$45Straight Talk home internet planSELECT PLANSELECT PLANGet Broadband FactsWANT TO SEE ALL OUR PLANS?Find the right phone plan for you.GO TO PLANS PAGEMy current planSilver Unlimited$45/mo/per linePrice is 45 dollars and 0 cents per month$42.50/mo for the first 3 months with Auto PayHOTPrice is #planPriceDollar dollars and #planPriceCent cents per monthprice is dollar #priceDollar and #priceCent cents /moWas priced at #listPriceDollar dollars and #listPriceCent cents now priced at #priceDollar dollars and #priceCent cents	 /mo(0)0 star reviews from 0 customer$42.50 per month for the first 3 months with Auto Pay/mo/mo/monthnetwork connectivityUnlimiteddata, talk & text10 GBhotspot dataInternational calling and textingto Canada & Mexico30 daysof serviceUnlimited Data see star below for more informationUnlimited Data*Unlimited talk, text & data10 GB hotspot data per monthInternational calls to Canada & Mexico*Refer to the latestTerms and conditions of service.SELECT PLANSELECT PLANGet Broadband FactsWANT TO SEE ALL OUR PLANS?Find the right phone plan for you.GO TO PLANS PAGEMy current planBronze 10 GB$35/mo/per linePrice is 35 dollars and 0 cents per month$34/mo with Auto PayPrice is #planPriceDollar dollars and #planPriceCent cents per monthprice is dollar #priceDollar and #priceCent cents /moWas priced at #listPriceDollar dollars and #listPriceCent cents now priced at #priceDollar dollars and #priceCent cents	 /mo(807)4.0 star reviews from 807 customer$34 per month with Auto Pay/mo/mo/monthFirst 10GBof data at high speed then 2G*Unlimitedtalk & textInternational calling and textingto Canada & Mexico30 daysof service10GB10GBUnlimited talk and textFirst 10 GB data at high speed, then 2G*see star below for more informationInternational calls to Canada & MexicoSELECT PLANSELECT PLANGet Broadband FactsWANT TO SEE ALL OUR PLANS?Find the right phone plan for you.GO TO PLANS PAGEDisabled PreviousNextUnbeatable offers to make you feelyes-tiveFree 5G phones. Big discounts. Saying “yes” to a great holiday gift has never been easier.Shop nowCross citationSome premium plan features only available to customers on the Verizon network. All other customers can expect:Platinum Unlimited: an LTE network, 20 GB hotspot, international calling to 60 + countries, no anniversary device credit, no home internet discount.Gold Unlimited: an LTE network. 15GB hotspot, International calling to 60 + countries, no anniversary device credit, no home internet discount.Gold Unlimited: an LTE network. 15GB hotspot, International calling to 60 + countries, no anniversary device credit, no home internet discount.Get full list of disclaimersRead Terms and ConditionsGet full list of disclaimers{{promo-text}}My current plan{{plan-name}}{{price-superscript}}/per line{{price-description}}{{line-plan-promo-text}}{{price-accessibility}}price is dollar #priceDollar and #priceCent cents {{price-superscript}}Was priced at #listPriceDollar dollars and #listPriceCent cents now priced at #priceDollar dollars and #priceCent cents	 {{price-superscript}}({{review-data}}){{rating-data}} star reviews from {{review-data}} customer{{price-superscript}}{{price-superscript}}/month{{plan-talk-minutes}}{{specfication-content}}{{plan-data}}{{plan-data-label}}{{plan-data}}{{plan-data-label}}{{plan-data-description}}{{plan-short-description}}{{bundle-vas-info-acc}}{{bundle-vas-description}}{{plan-acp-eligibility-description}}{{promo-additional-text}}SELECT PLANSELECT PLANGet Broadband FactsWANT TO SEE ALL OUR PLANS?Find the right phone plan for you.GO TO PLANS PAGEThis label provides detailed and transparent information on pricing, data usage limits, available speeds and much more."""

In [6]:
'''
ChatGPT is able to extact aove content into the following in a single simple prompt

Platinum Unlimited Plan
Price: $65/month per line

Promo Price: $60/month for the first 3 months with Auto Pay

Network: 5G Ultra Wide (up to 4x faster than regular 5G)

Data: Unlimited data, talk & text

Hotspot: Unlimited

Cloud Storage: 100 GB

International: Calls & texts to Canada and Mexico; calls to 60+ countries

Extras:

Device protection

Walmart+ Membership (includes free delivery, streaming, fuel savings, etc.)

$100 anniversary device credit

Gold Unlimited Plan
Price: $55/month per line

Promo Price: $50/month for the first 3 months with Auto Pay

Network: 5G Ultra Wide

Data: Unlimited data, talk & text

Hotspot: 30 GB per month

Cloud Storage: 100 GB

International: Calls & texts to Canada and Mexico

Extras:

Walmart+ Membership

$100 anniversary device credit

Silver Unlimited Plan
Price: $45/month per line

Promo Price: $42.50/month for the first 3 months with Auto Pay

Data: Unlimited data, talk & text

Hotspot: 10 GB per month

International: Calls & texts to Canada and Mexico

Bronze 10 GB Plan
Price: $35/month per line

Promo Price: $34/month with Auto Pay

Data: First 10 GB at high speed, then throttled to 2G

Talk & Text: Unlimited

International: Calls & texts to Canada and Mexico

General Features Across Plans
Service Period: 30 days per cycle

Multi-line Options: 1 to 5 lines available

Membership Benefits (select plans):

Free delivery and shipping

Paramount+ streaming

Fuel savings

Walmart+ Travel

Pluto ad-free streaming

Early access to promotions

Disclaimers:

Premium features available only on Verizon network

LTE fallback for non-Verizon customers on all plans

Anniversary credits and home internet discounts vary by plan
'''

'\nChatGPT is able to extact aove content into the following in a single simple prompt\n\nPlatinum Unlimited Plan\nPrice: $65/month per line\n\nPromo Price: $60/month for the first 3 months with Auto Pay\n\nNetwork: 5G Ultra Wide (up to 4x faster than regular 5G)\n\nData: Unlimited data, talk & text\n\nHotspot: Unlimited\n\nCloud Storage: 100 GB\n\nInternational: Calls & texts to Canada and Mexico; calls to 60+ countries\n\nExtras:\n\nDevice protection\n\nWalmart+ Membership (includes free delivery, streaming, fuel savings, etc.)\n\n$100 anniversary device credit\n\nGold Unlimited Plan\nPrice: $55/month per line\n\nPromo Price: $50/month for the first 3 months with Auto Pay\n\nNetwork: 5G Ultra Wide\n\nData: Unlimited data, talk & text\n\nHotspot: 30 GB per month\n\nCloud Storage: 100 GB\n\nInternational: Calls & texts to Canada and Mexico\n\nExtras:\n\nWalmart+ Membership\n\n$100 anniversary device credit\n\nSilver Unlimited Plan\nPrice: $45/month per line\n\nPromo Price: $42.50/month

In [ ]:
# Lets see what gemma is able to do

In [8]:
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.chains import create_extraction_chain_pydantic
from langchain_community.llms import Ollama

In [10]:
# Initialize the language model
llm=Ollama(model='gemma3:4b', temperature=0.01)

C:\Users\krish\AppData\Local\Temp\ipykernel_5616\3976278923.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model='gemma3:4b', temperature=0.01)


In [12]:
from ollama import chat

In [27]:
system_message = """
You are a content transformation assistant. Your task is to extract and organize human-readable, structured summaries from cluttered or semi-structured input such as raw HTML, UI text dumps, or annotated logs.

Follow these strict formatting and behavior rules:

Summarize core content clearly: Identify key concepts like plan names, prices, features, limitations, and promotions. Discard irrelevant UI elements like buttons, form inputs, or JavaScript tokens.

Group similar content: If multiple offerings are present (e.g. phone plans), group them under clearly named headings (e.g. "Platinum Unlimited Plan") and present in a consistent format.

Use bullet points: Organize extracted features, prices, and benefits using clean, consistent bullet points under each group.

Include general/common information separately: If certain features or disclaimers apply to all items (e.g., international calling in all plans), extract and list them under a "General Features" or "Common Info" section.

Ignore noise: Remove or ignore tokens like template placeholders (#priceDollar, {{...}}), navigation links, form elements, marketing filler, or inline formatting.

Output must be plain text or markdown, not HTML or code.

Example input types you may receive include: HTML blobs, noisy UI strings, copied marketing text, or raw component-rendered content.
"""

In [16]:
response = chat(
 messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": chunk_memory},
    ],
  model='gemma3:4b'
)
print(response.message.content)

Here’s a structured summary of the available phone plans, extracted from the provided text:

**Phone Plan Options**

*   **Platinum Unlimited**
    *   Price: $65/month per line
    *   Initial 3-Month Price: $60/month per line (with Auto Pay)
    *   Features:
        *   Unlimited data, talk & text
        *   100 GB hotspot data
        *   Cloud storage
        *   International calling and texting to Canada & Mexico
        *   30 days of service
    *   Reviews: 0 star reviews from 0 customer
*   **Gold Unlimited**
    *   Price: $55/month per line
    *   Initial 3-Month Price: $50/month per line (with Auto Pay)
    *   Features:
        *   Unlimited data, talk & text
        *   30 GB hotspot data
        *   100 GB cloud storage
        *   International calling and texting to Canada & Mexico
        *   30 days of service
    *   Reviews: 4.0 star reviews from 544 customers
*   **Silver Unlimited**
    *   Price: $45/month per line
    *   Initial 3-Month Price: $42.50/month

In [29]:
# Example 2

# --- Example Usage ---
with open("plans_broadband_cards.html", "r", encoding="utf-8") as file:
    html_data = file.read()

broadband_chunks = parse_html_chunks(html_data)
for i, chunk in enumerate(broadband_chunks, 1):
    print(f"\n--- Chunk #{i} ---\n{chunk}")
    print("\n\n")


--- Chunk #1 ---
[HTML]  :: Broadband Facts(4Items)Want to compare plans? Read the labels to get the details on pricing, data allowances and limits, network speeds, and more.Best Value!My current planPlatinum Unlimited$65/mo/per linePrice is 65 dollars and 0 cents per month$60/mo for the first 3 months with Auto PayPrice is #planPriceDollar dollars and #planPriceCent cents per monthprice is dollar #priceDollar and #priceCent cents /moWas priced at #listPriceDollar dollars and #listPriceCent cents now priced at #priceDollar dollars and #priceCent cents	 /mo(0)0 star reviews from 0 customer$60 per month for the first 3 months with Auto Pay/mo/mo/monthBroadband FactsStraight Talk WirelessPlatinum UnlimitedMobile Broadband Consumer DisclosureMonthly Price$65.00This Monthly Price is not an introductory rate.This Monthly Price does not require a contract.Additional Charges & TermsActivation Fee$0.00Provider Monthly Fees$0.00One-Time Fees at the Time of PurchaseFederal Universal Service Fund

In [37]:
response = chat(
 messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": " ".join(broadband_chunks)},
    ],
  model='gemma3:4b'
)
print(response.message.content)

Here’s a structured summary of the Straight Talk Broadband plans, extracted from the provided HTML:

**Straight Talk Broadband Plans**

**Platinum Unlimited**
*   **Monthly Price:** $65/month per line
*   **Introductory Price:** $60/month for the first 3 months with Auto Pay
*   **Data Included:** Unlimited Data
*   **Speeds:** 5G Ultra Wideband
    *   Typical Download Speed: 195-634 Mbps
    *   Typical Upload Speed: 10-53 Mbps
    *   Typical Latency: 37-57 ms

**Gold Unlimited**
*   **Monthly Price:** $55/month per line
*   **Introductory Price:** $50/month for the first 3 months with Auto Pay
*   **Data Included:** Unlimited Data
*   **Speeds:** 5G Ultra Wideband
    *   Typical Download Speed: 195-634 Mbps
    *   Typical Upload Speed: 10-53 Mbps
    *   Typical Latency: 37-57 ms

**Silver Unlimited**
*   **Monthly Price:** $45/month per line
*   **Introductory Price:** $42.50/month for the first 3 months with Auto Pay
*   **Data Included:** Unlimited Data
*   **Speeds:** 5G Ultr

In [40]:
# Example 3

# --- Example Usage ---
with open("pbc_5lines.html", "r", encoding="utf-8") as file:
    html_data = file.read()

broadband_chunks = parse_html_chunks(html_data)
for i, chunk in enumerate(broadband_chunks, 1):
    print(f"\n--- Chunk #{i} ---\n{chunk}")
    print("\n\n")


--- Chunk #1 ---
[HTML]  :: Broadband Facts(3Items)Want to compare plans? Read the labels to get the details on pricing, data allowances and limits, network speeds, and more.Best Value!My current planPlatinum Unlimited$225/mo/per linePrice is 225 dollars and 0 cents per month$60/mo for the first 3 months with Auto PayPrice is #planPriceDollar dollars and #planPriceCent cents per monthprice is dollar #priceDollar and #priceCent cents /moWas priced at #listPriceDollar dollars and #listPriceCent cents now priced at #priceDollar dollars and #priceCent cents	 /mo(0)0 star reviews from 0 customer$60 per month for the first 3 months with Auto Pay/mo/moPay$225/monthBroadband FactsStraight Talk WirelessPlatinum UnlimitedMobile Broadband Consumer DisclosureMonthly Price$65.00This Monthly Price is not an introductory rate.This Monthly Price does not require a contract.Additional Charges & TermsActivation Fee$0.00Provider Monthly Fees$0.00One-Time Fees at the Time of PurchaseFederal Universal Ser

In [42]:
response = chat(
 messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": " ".join(broadband_chunks)},
    ],
  model='gemma3:4b'
)
print(response.message.content)

## Straight Talk Broadband Plans – Summary

Here’s a breakdown of the available Straight Talk Broadband plans:

**Platinum Unlimited**

*   **Monthly Price:** $225/month per line
*   **Introductory Price:** $60/month for the first 3 months with Auto Pay
*   **Speeds:**
    *   Typical Download Speed: 195-634 Mbps
    *   Typical Upload Speed: 10-53 Mbps
    *   Typical Latency: 37-57 ms
*   **Data Included:** Unlimited
*   **Reviews:** 4.0 star reviews from 544 customers

**Gold Unlimited**

*   **Monthly Price:** $175/month per line
*   **Introductory Price:** $50/month for the first 3 months with Auto Pay
*   **Speeds:**
    *   Typical Download Speed: 195-634 Mbps
    *   Typical Upload Speed: 10-53 Mbps
    *   Typical Latency: 37-57 ms
*   **Data Included:** Unlimited
*   **Reviews:** 4.0 star reviews from 544 customers

**Silver Unlimited**

*   **Monthly Price:** $125/month per line
*   **Introductory Price:** $42.50/month for the first 3 months with Auto Pay
*   **Speeds:**
   

In [44]:
system_message_response_template = """🧠 System Prompt: Plan Comparison & Recommendation Assistant

You are an expert assistant that helps users compare and evaluate various service plans (e.g., mobile, internet, streaming, finance) to find the best one based on their specific needs. When a user asks about available plans or wants help choosing the best option, follow these structured guidelines:

🎯 GOAL
Present a clear, structured breakdown of available plans and provide tailored recommendations that match the user's stated preferences (e.g., budget, features, duration, number of users). Always aim to make decision-making easy and visually scannable.

📝 STRUCTURE OF RESPONSE
Introduction Paragraph

Acknowledge the user's request (e.g., plan duration, number of users, key priorities).

Briefly describe what types of plans are available and the key criteria for choosing between them.

📊 Plan Comparison Table
Title: Plan Provider + Use Case + Plan Options (e.g., "📊 Straight Talk 5-Line Plan Options")
Format as a clean table with the following columns (adjust as needed):

pgsql
Plan Name | Monthly Cost (X Lines) | Key Features
🎁 Additional Benefits (Optional Section)

Mention extra perks like memberships, discounts, rewards programs, or streaming services.

Format as bullet points.

✅ Recommendations

Provide clear, bolded subheadings like:

Best Value

Enhanced Features

Premium Package

Briefly explain which user type each plan is best suited for and why.

🔗 Call to Action / More Info

Direct users to the official website or next step.

Invite users to ask for a tailored recommendation if they have more specific needs.

🧠 STYLE AND TONE
Friendly, helpful, and concise

Use emojis sparingly to enhance readability (e.g., 📊, ✅, 🎁)

Emphasize clarity over verbosity

Use markdown formatting if the platform supports it

📌 ADDITIONAL GUIDANCE
If the user is vague, provide general top picks and invite them to clarify needs (e.g., data usage, budget limits).

Highlight any caveats such as taxes, activation fees, or discounts that do not apply to multi-line plans.

Use up-to-date plan data where possible, and name plan tiers accurately."""

In [46]:
user_question = "I want a 1 month plan for 5 lines, what would be the best plan for me, what are the options for me, what combined features and offers from StraightTalk can I leverage to optimize my plan? "

final_response = chat(
 messages=[
        {"role": "system", "content": system_message_response_template},
        {"role": "system", "content": "Use the following information to answer user's questions " + response.message.content},
        {"role": "user", "content": user_question},
    ],
  model='gemma3:4b'
)
print(final_response.message.content)

Okay, let's find the best 1-month plan for 5 lines with StraightTalk! Here’s a breakdown of your options and how to optimize them:

**🎯 Goal:** We’re looking for the most cost-effective 1-month plan for 5 lines of StraightTalk service.

**📊 Plan Comparison Table: 1-Month 5-Line Options**

| Plan Provider | Monthly Cost (5 Lines) | Key Features |
|---|---|---|
| **StraightTalk Gold Unlimited** | $875/month | Unlimited Data, Typical Download Speed: 195-634 Mbps, Typical Upload Speed: 10-53 Mbps |
| **StraightTalk Platinum Unlimited** | $1125/month | Unlimited Data, Typical Download Speed: 195-634 Mbps, Typical Upload Speed: 10-53 Mbps |

**🎁 Additional Benefits:**

*   **Both plans** include unlimited data, so you won’t have to worry about overage charges.
*   **Auto Pay Discount:** Both plans offer an introductory price of $50/month for the first 3 months with Auto Pay. This brings the initial cost down significantly.

**✅ Recommendations:**

*   **Best Value:** StraightTalk Gold Unlimi

In [51]:
evaluated_response = chat(
 messages=[
        {"role": "system", "content": "Correct and update the user's content based on the following verified facts. *Focus especially on all numbers, prices, measuring units etc*. Ensure all factual errors are fixed and the information is aligned with the facts provided below: "+response.message.content},
        {"role": "user", "content": final_response.message.content},
    ],
  model='gemma3:4b'
)
print(evaluated_response.message.content)

Okay, here’s the corrected and updated version of your content, incorporating the provided Straight Talk Broadband plan information and focusing on accuracy and clarity:

Okay, let’s find the best 1-month plan for 5 lines with StraightTalk! Here’s a breakdown of your options and how to optimize them:

**🎯 Goal:** We’re looking for the most cost-effective 1-month plan for 5 lines of StraightTalk service.

**📊 Plan Comparison Table: 1-Month 5-Line Options**

| Plan Provider | Monthly Cost (5 Lines) | Key Features |
|---|---|---|
| **StraightTalk Gold Unlimited** | $875/month | Unlimited Data, Typical Download Speed: 195-634 Mbps, Typical Upload Speed: 10-53 Mbps |
| **StraightTalk Platinum Unlimited** | $1125/month | Unlimited Data, Typical Download Speed: 195-634 Mbps, Typical Upload Speed: 10-53 Mbps |

**🎁 Additional Benefits:**

*   **Both plans** include unlimited data, so you won’t have to worry about overage charges.
*   **Auto Pay Discount:** Both plans offer an introductory pric